In [62]:
#!/usr/bin/env python3
#
# FileSender www.filesender.org
#
# Copyright (c) 2009-2019, AARNet, Belnet, HEAnet, SURFnet, UNINETT
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
# *   Redistributions of source code must retain the above copyright
#     notice, this list of conditions and the following disclaimer.
# *   Redistributions in binary form must reproduce the above copyright
#     notice, this list of conditions and the following disclaimer in the
#     documentation and/or other materials provided with the distribution.
# *   Neither the name of AARNet, Belnet, HEAnet, SURFnet and UNINETT nor the
#     names of its contributors may be used to endorse or promote products
#     derived from this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
# DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
# FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
# DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
# SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
# CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
# OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
# OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.


In [1]:
import argparse
import requests
import time
from collections.abc import Iterable
from collections.abc import MutableMapping
import hmac
import hashlib
import urllib3
import os
import json
import configparser
from os.path import expanduser


In [2]:
from multiprocessing import Pool
from functools import partial
import time

In [3]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


In [4]:
def flatten(d, parent_key=''):
    items = []
    for k, v in d.items():
        new_key = parent_key + '[' + k + ']' if parent_key else k
        if isinstance(v, MutableMapping):
            items.extend(flatten(v, new_key).items())
        else:
            items.append(new_key+'='+v)
    items.sort()
    return items



In [5]:

def call(method, path, data, content=None, rawContent=None, options={}):
    data['remote_user'] = username
    data['timestamp'] = str(round(time.time()))
    flatdata = flatten(data)
    signed = bytes(method+'&'+base_url.replace('https://', '',
                   1).replace('http://', '', 1)+path+'?'+('&'.join(flatten(data))), 'ascii')

    content_type = options['Content-Type'] if 'Content-Type' in options else 'application/json'

    inputcontent = None
    if content is not None and content_type == 'application/json':
        inputcontent = json.dumps(content, separators=(',', ':'))
        signed += bytes('&'+inputcontent, 'ascii')
    elif rawContent is not None:
        inputcontent = rawContent
        signed += bytes('&', 'ascii')
        signed += inputcontent

    # print(signed)
    bkey = bytearray()
    bkey.extend(map(ord, apikey))
    data['signature'] = hmac.new(bkey, signed, hashlib.sha1).hexdigest()

    url = base_url+path+'?'+('&'.join(flatten(data)))
    headers = {
        "Accept": "application/json",
        "Content-Type": content_type
    }
    response = None
    if method == "get":
        response = requests.get(url, verify=not insecure, headers=headers)
    elif method == "post":
        response = requests.post(
            url, data=inputcontent, verify=not insecure, headers=headers)
    elif method == "put":
        response = requests.put(url, data=inputcontent,
                                verify=not insecure, headers=headers)
    elif method == "delete":
        response = requests.delete(url, verify=not insecure, headers=headers)

    if response is None:
        raise Exception('Client error')

    code = response.status_code
    # print(url)
    # print(inputcontent)
    # print(code)
    # print(response.text)

    if code != 200:
        if method != 'post' or code != 201:
            raise Exception('Http error '+str(code)+' '+response.text)

    if response.text == "":
        raise Exception('Http error '+str(code)+' Empty response')

    if method != 'post':
        return response.json()

    r = {}
    r['location'] = response.headers['Location']
    r['created'] = response.json()
    return r


In [6]:

def postTransfer(user_id, files, recipients, subject=None, message=None, expires=None, options=[]):

    if expires is None:
        expires = round(time.time()) + (default_transfer_days_valid*24*3600)

    to = [x.strip() for x in recipients.split(',')]
    print(to)

    return call(
        'post',
        '/transfer',
        {},
        {
            'from': user_id,
            'files': files,
            'recipients': to,
            'subject': subject,
            'message': message,
            'expires': expires,
            'aup_checked': 1,
            'options': options
        },
        None,
        {}
    )


In [7]:
def putChunk(t, f, chunk, offset):
    return call(
        'put',
        '/file/'+str(f['id'])+'/chunk/'+str(offset),
        {'key': f['uid'], 'roundtriptoken': t['roundtriptoken']},
        None,
        chunk,
        {'Content-Type': 'application/octet-stream'}
    )



In [8]:
def fileComplete(t,f):
    return call(
        'put',
        '/file/'+str(f['id']),
        { 'key': f['uid'], 'roundtriptoken': t['roundtriptoken'] },
        { 'complete': True },
        None,
        {}
    )


In [9]:


def deleteTransfer(transfer):
    return call(
        'delete',
        '/transfer/'+str(transfer['id']),
        {'key': transfer['files'][0]['uid']},
        None,
        None,
        {}
    )

In [10]:

def transferComplete(transfer):
    return call(
        'put',
        '/transfer/'+str(transfer['id']),
        {'key': transfer['files'][0]['uid']},
        {'complete': True},
        None,
        {}
    )

In [11]:
def upload_file( fileobject, transferData, filesData, upload_chunk_size, debug):
    fname = fileobject["name"]
    fsize = fileobject["size"]
    fstring = f"{fname}:{fsize}"
    fpath = filesData[fstring]["path"]

    try:
        # putChunks
        if debug:
            print('putChunks: '+fpath)
        with open(fpath, mode='rb', buffering=0) as fin:
            chunk_count = 0
            for offset in range(0, fsize, upload_chunk_size):
                if progress:
                    print('Uploading: '+fpath+' '+str(offset)+'-'+str(min(offset +
                          upload_chunk_size, fsize))+' '+str(round(offset/fsize*100))+'%')
                data = fin.read(upload_chunk_size)
                # print(data)
                putChunk(transferData, fileobject, data, offset)
                if debug:
                    chunk_count += 1
                    print(f"uploaded {chunk_count} chunks")
        # file complete
        if debug:
            print('fileComplete: '+fpath)
        res = fileComplete(transferData, fileobject)
        if progress:
            print('Uploading: '+fpath+' '+str(size)+' 100%')
        return res
    except Exception as e:
        raise(e)


In [29]:
def transfer_data_to_text(tdata):
    total_size = 0
    for f in tdata["files"]:
        total_size += f["size"]
    size_unit = "B"
    if total_size > 1024**3:
        size_unit = "GB"
        total_size = total_size/(1024**3)
    elif total_size > 1024**2:
        size_unit = "MB"
        total_size = total_size/(1024**2)
    elif total_size > 1024:
        size_unit = "kB"
        total_size = total_size/(1024)
    size_str = f"{total_size:.2f} {size_unit}"
    url = res["recipients"][0]["download_url"]
    report_txt = f"""FileSender upload
transfer ID: {tdata["id"]}
uploaded by: {tdata["user_email"]}
total size:  {size_str}

recipient:   {tdata["recipients"][0]["email"]}
D/L link:    {url}

Files uploaded:
"""

    for fidx, f in enumerate(tdata["files"]):
        url = f"https://filesender.aarnet.edu.au/download.php?token={res['recipients'][0]['token']}&files_ids={f['id']}"
        ct_str = f"{fidx}".rjust( len(str(len(tdata["files"]))) )
        report_txt += f"""
{ct_str}: "{f["name"]}"
{" "*len(ct_str)}  {f["size"]:,} bytes 
{" "*len(ct_str)}  {url}&files_ids={f["id"]}
"""

    return report_txt



In [13]:

#argv
parser = argparse.ArgumentParser()
parser.add_argument("files", help="path to file(s) to send", nargs='+')
parser.add_argument("-v", "--verbose", action="store_true")
parser.add_argument("-i", "--insecure", action="store_true")
parser.add_argument("-p", "--progress", action="store_true")
parser.add_argument("-s", "--subject")
parser.add_argument("-m", "--message")
parser.add_argument("-k", "--skip-email", action="store_true", default=False, help="Don't send email to recipient")

requiredNamed = parser.add_argument_group('required named arguments')



In [14]:
n_procs = 4

In [15]:
n_procs = 4

filelist = """/home/paul/Zotero/storage/28F8IG85/Lokody - 2013 - Cancer genomics Signature analysis suggests cance.pdf
/home/paul/Zotero/storage/29G6QV82/Usuyama et al. - 2014 - HapMuC somatic mutation calling using heterozygou.pdf
/home/paul/Zotero/storage/2BBJ65XW/Pleasance et al. - 2010 - A small cell lung cancer genome reports complex to.pdf
/home/paul/Zotero/storage/2CR2WGUC/Taub et al. - 2010 - Overcoming bias and systematic errors in next gene.pdf
/home/paul/Zotero/storage/2DRPFWV8/Kim et al. - 2016 - Somatic ERCC2 mutations are associated with a dist.pdf
/home/paul/Zotero/storage/2FXQNZ3S/Alter et al. - 2000 - Singular value decomposition for genome-wide expre.pdf
/home/paul/Zotero/storage/2GTA3FY6/Garcia-Prieto et al. - 2020 - The consequences of variant calling decisions in s.pdf
/home/paul/Zotero/storage/2JRP4NTD/Kim et al. - 2018 - Strelka2 fast and accurate calling of germline an.pdf
/home/paul/Zotero/storage/2P6RPSFQ/Pellegrino et al. - 2018 - High-throughput single-cell DNA sequencing of acut.pdf
/home/paul/Zotero/storage/2QNXMWPI/Kandoth et al. - 2013 - Mutational landscape and significance across 12 ma.pdf
/home/paul/Zotero/storage/2S6DPCZ7/Liu et al. - 2016 - Joint detection of copy number variations in paren.pdf
/home/paul/Zotero/storage/2UA83RGN/Consortium - 2012 - An integrated map of genetic variation from 1,092 .pdf
/home/paul/Zotero/storage/333VUI79/Zhu-SII09.pdf
/home/paul/Zotero/storage/34C3ERMA/Bersanelli et al. - 2016 - Methods for the integration of multi-omics data m.pdf
/home/paul/Zotero/storage/3BGPU5KB/Hansen et al. - 2012 - BSmooth from whole genome bisulfite sequencing re.pdf
/home/paul/Zotero/storage/3CFDCRFR/O'Brien et al. - 2010 - Gremlin An Interactive Visualization Model for An.pdf
/home/paul/Zotero/storage/3HBZH2XM/gim2014178a.pdf
/home/paul/Zotero/storage/3HMKCBF3/Parisi et al. - 2014 - Ranking and combining multiple predictors without .pdf
/home/paul/Zotero/storage/3IWJ9K27/Rabbani et al. - 2014 - The promise of whole-exome sequencing in medical g.pdf
/home/paul/Zotero/storage/3JNN5N2W/Pengelly et al. - 2013 - A SNP profiling panel for sample tracking in whole.pdf""".splitlines()

n_procs = min(n_procs, len(filelist))

In [16]:
# settings
base_url = 'https://filesender.aarnet.edu.au/rest.php'
default_transfer_days_valid = 21
username = None
apikey = None
homepath = expanduser("~")

config = configparser.ConfigParser()
config.read(homepath + '/.filesender/filesender.py.ini')
if 'system' in config:
    base_url = config['system'].get(
        'base_url', 'https://filesender.aarnet.edu.au/rest.php')
    default_transfer_days_valid = int(
        config['system'].get('default_transfer_days_valid', 10))
if 'user' in config:
    username = config['user'].get('username')
    apikey = config['user'].get('apikey')

# if we have found these in the config file they become optional arguments
if username is None:
    requiredNamed.add_argument("-u", "--username", required=True)
else:
    parser.add_argument("-u", "--username")

if apikey is None:
    requiredNamed.add_argument("-a", "--apikey", required=True)
else:
    parser.add_argument("-a", "--apikey")

requiredNamed.add_argument("-r", "--recipients", required=True)
# args = parser.parse_args("-p -r paul.wang@sahmri.com,paul.wang@sa-genomics.com.au -v -s subject -m messagebody".split(" ") + filelist)
args = parser.parse_args("-p -r paul.wang@sahmri.com -v -s subject -m messagebody".split(" ") + filelist)
debug = args.verbose
progress = args.progress
insecure = args.insecure
skip_email = args.skip_email


if args.username is not None:
    username = args.username

if args.apikey is not None:
    apikey = args.apikey



In [17]:
base_url = 'https://filesender.aarnet.edu.au/rest.php'
response = requests.get(base_url+'/info', verify=True)
response.json()


{'url': 'https://filesender.aarnet.edu.au/',
 'name': 'AARNet FileSender',
 'logon_url': 'https://cloudstor.aarnet.edu.au/simplesaml/module.php/core/as_login.php?AuthId=default-sp&ReturnTo=__target__',
 'upload_chunk_size': 12582912}

In [18]:

# configs
try:
    response = requests.get(base_url+'/info', verify=True)
except requests.exceptions.SSLError as exc:
    if not insecure:
        print('Error: the SSL certificate of the server you are connecting to cannot be verified:')
        print(exc)
        print('For more information, please refer to https://www.digicert.com/ssl/. If you are absolutely certain of the identity of the server you are connecting to, you can use the --insecure flag to bypass this warning. Exiting...')
        sys.exit(1)
    elif insecure:
        print('Warning: Error: the SSL certificate of the server you are connecting to cannot be verified:')
        print(exc)
        print('Running with --insecure flag, ignoring warning...')
        response = requests.get(base_url+'/info', verify=False)
upload_chunk_size = response.json()['upload_chunk_size']

if debug:
    print('base_url          : '+base_url)
    print('username          : '+username)
    print('apikey            : '+apikey)
    print('upload_chunk_size : '+str(upload_chunk_size)+' bytes')
    print('recipients        : '+args.recipients)
    print('files             : '+','.join(args.files))
    print('insecure          : '+str(insecure))



base_url          : https://filesender.aarnet.edu.au/rest.php
username          : paul.wang@adelaide.edu.au
apikey            : d1f80e881bb96605a1e903b6647330e7722b43bdd5385a09e3eb97ce9bd4ea19
upload_chunk_size : 12582912 bytes
recipients        : paul.wang@sahmri.com
files             : /home/paul/Zotero/storage/28F8IG85/Lokody - 2013 - Cancer genomics Signature analysis suggests cance.pdf,/home/paul/Zotero/storage/29G6QV82/Usuyama et al. - 2014 - HapMuC somatic mutation calling using heterozygou.pdf,/home/paul/Zotero/storage/2BBJ65XW/Pleasance et al. - 2010 - A small cell lung cancer genome reports complex to.pdf,/home/paul/Zotero/storage/2CR2WGUC/Taub et al. - 2010 - Overcoming bias and systematic errors in next gene.pdf,/home/paul/Zotero/storage/2DRPFWV8/Kim et al. - 2016 - Somatic ERCC2 mutations are associated with a dist.pdf,/home/paul/Zotero/storage/2FXQNZ3S/Alter et al. - 2000 - Singular value decomposition for genome-wide expre.pdf,/home/paul/Zotero/storage/2GTA3FY6/Garcia-Pr

In [19]:

# postTransfer
if debug:
    print('postTransfer')

files = {}
filesTransfer = []
for f in args.files:
    fn_abs = os.path.abspath(f)
    fn = os.path.basename(fn_abs)
    size = os.path.getsize(fn_abs)

    files[fn+':'+str(size)] = {
        'name': fn,
        'size': size,
        'path': fn_abs
    }
    filesTransfer.append({'name': fn, 'size': size})

troptions = {'get_a_link': skip_email}

# sort by decreasing file size
filesTransfer = sorted(filesTransfer, key=lambda x: x["size"], reverse=True)

transfer = postTransfer(username,
                        filesTransfer,
                        args.recipients,
                        subject=args.subject,
                        message=args.message,
                        expires=None,
                        options=troptions)['created']
# print(transfer)


postTransfer
['paul.wang@sahmri.com']


In [20]:
# transfer

In [21]:
t0 = time.time()

n_procs = 1
task = partial(upload_file, 
               transferData=transfer, 
               filesData=files,
               upload_chunk_size=upload_chunk_size,
               debug=debug)

pool = Pool(n_procs)
results = pool.map(task, transfer['files'])
pool.close()

t1 = time.time()
print(t1-t0)

putChunks: /home/paul/Zotero/storage/2QNXMWPI/Kandoth et al. - 2013 - Mutational landscape and significance across 12 ma.pdf
Uploading: /home/paul/Zotero/storage/2QNXMWPI/Kandoth et al. - 2013 - Mutational landscape and significance across 12 ma.pdf 0-8826624 0%
uploaded 1 chunks
fileComplete: /home/paul/Zotero/storage/2QNXMWPI/Kandoth et al. - 2013 - Mutational landscape and significance across 12 ma.pdf
Uploading: /home/paul/Zotero/storage/2QNXMWPI/Kandoth et al. - 2013 - Mutational landscape and significance across 12 ma.pdf 627990 100%
putChunks: /home/paul/Zotero/storage/3HBZH2XM/gim2014178a.pdf
Uploading: /home/paul/Zotero/storage/3HBZH2XM/gim2014178a.pdf 0-5283692 0%
uploaded 1 chunks
fileComplete: /home/paul/Zotero/storage/3HBZH2XM/gim2014178a.pdf
Uploading: /home/paul/Zotero/storage/3HBZH2XM/gim2014178a.pdf 627990 100%
putChunks: /home/paul/Zotero/storage/2P6RPSFQ/Pellegrino et al. - 2018 - High-throughput single-cell DNA sequencing of acut.pdf
Uploading: /home/paul/Zotero/sto

In [22]:
# transferComplete
if debug:
    print('transferComplete')
res = transferComplete(transfer)
if progress:
    print('Upload Complete')


transferComplete
Upload Complete


In [23]:
# res

In [132]:
fout = open("./test/final_dump.json", "w")
fout.write(json.dumps(res))
fout.close()

In [30]:
print(transfer_data_to_text(res))

FileSender upload
transfer ID: 3137421
uploaded by: paul.wang@adelaide.edu.au
total size:  34.93 MB

recipient:   paul.wang@sahmri.com
D/L link:    https://filesender.aarnet.edu.au/?s=download&token=2c3d7a65-6408-4ad0-827f-cdd096d8cee5

Files uploaded:

 0: "Kandoth et al. - 2013 - Mutational landscape and significance across 12 ma.pdf"
    8,826,624 bytes 
    https://filesender.aarnet.edu.au/download.php?token=2c3d7a65-6408-4ad0-827f-cdd096d8cee5&files_ids=21052951&files_ids=21052951

 1: "gim2014178a.pdf"
    5,283,692 bytes 
    https://filesender.aarnet.edu.au/download.php?token=2c3d7a65-6408-4ad0-827f-cdd096d8cee5&files_ids=21052956&files_ids=21052956

 2: "Pellegrino et al. - 2018 - High-throughput single-cell DNA sequencing of acut.pdf"
    3,988,467 bytes 
    https://filesender.aarnet.edu.au/download.php?token=2c3d7a65-6408-4ad0-827f-cdd096d8cee5&files_ids=21052961&files_ids=21052961

 3: "Garcia-Prieto et al. - 2020 - The consequences of variant calling decisions in s.pdf"
 

In [31]:
res

{'id': 3137421,
 'userid': 76641,
 'user_email': 'paul.wang@adelaide.edu.au',
 'subject': 'subject',
 'message': 'messagebody',
 'created': {'raw': 1699503146, 'formatted': '09/11/2023'},
 'expires': {'raw': 1701317535, 'formatted': '30/11/2023'},
 'expiry_date_extension': 0,
 'options': {'get_a_link': False,
  'add_me_to_recipients': False,
  'email_recipient_when_transfer_expires': True,
  'encryption': False},
 'salt': 'EKuz3wknSXnUeswS0k53rga8SoYgWnxl',
 'roundtriptoken': '',
 'files': [{'id': 21052951,
   'transfer_id': 3137421,
   'uid': 'c09c2587-73f5-4c30-a0f0-16d3dce2b4d9',
   'name': 'Kandoth et al. - 2013 - Mutational landscape and significance across 12 ma.pdf',
   'size': 8826624,
   'sha1': None},
  {'id': 21052956,
   'transfer_id': 3137421,
   'uid': '11693a50-836c-40c3-a8ce-de4b9385dbce',
   'name': 'gim2014178a.pdf',
   'size': 5283692,
   'sha1': None},
  {'id': 21052961,
   'transfer_id': 3137421,
   'uid': '45bc05fd-87d3-4621-8b73-3102192fcbc1',
   'name': 'Pellegr

In [141]:
# text output

tdata = res

total_size = 0
for f in tdata["files"]:
    total_size += f["size"]
size_unit = "B"
if total_size > 1024**3:
    size_unit = "GB"
    total_size = total_size/(1024**3)
elif total_size > 1024**2:
    size_unit = "MB"
    total_size = total_size/(1024**2)
elif total_size > 1024:
    size_unit = "kB"
    total_size = total_size/(1024)

size_str = f"{total_size:.2f} {size_unit}"


url = res["recipients"][0]["download_url"]

report_txt = f"""FileSender upload
transfer ID: {tdata["id"]}
uploaded by: {tdata["user_email"]}
total size:  {size_str}

recipient:   {res["recipients"][0]["email"]}
D/L link:    {url}

Files uploaded:
"""

for fidx, f in enumerate(res["files"]):
    url = f"https://filesender.aarnet.edu.au/download.php?token={res['recipients'][0]['token']}&files_ids={f['id']}"
    ct_str = f"{fidx}".rjust( len(str(len(res["files"]))) )
    report_txt += f"""
{ct_str}: "{f["name"]}"
{" "*len(ct_str)}  {f["size"]:,} bytes 
{" "*len(ct_str)}  url: {url}&files_ids={f["id"]}
"""

# print(report_txt)





FileSender upload
transfer ID: 3137356
uploaded by: paul.wang@adelaide.edu.au
total size:  34.93 MB

recipient:   paul.wang@sahmri.com
D/L link:    https://filesender.aarnet.edu.au/?s=download&token=3aea5b1c-4ce6-4328-8d9a-1ccde0d2bf20

Files uploaded:

 0: "Kandoth et al. - 2013 - Mutational landscape and significance across 12 ma.pdf"
    8,826,624 bytes 
    url: https://filesender.aarnet.edu.au/download.php?token=3aea5b1c-4ce6-4328-8d9a-1ccde0d2bf20&files_ids=21052436&files_ids=21052436

 1: "gim2014178a.pdf"
    5,283,692 bytes 
    url: https://filesender.aarnet.edu.au/download.php?token=3aea5b1c-4ce6-4328-8d9a-1ccde0d2bf20&files_ids=21052441&files_ids=21052441

 2: "Pellegrino et al. - 2018 - High-throughput single-cell DNA sequencing of acut.pdf"
    3,988,467 bytes 
    url: https://filesender.aarnet.edu.au/download.php?token=3aea5b1c-4ce6-4328-8d9a-1ccde0d2bf20&files_ids=21052446&files_ids=21052446

 3: "Garcia-Prieto et al. - 2020 - The consequences of variant calling decisi

In [105]:
res["recipients"]

[{'id': 5455011,
  'transfer_id': 3137241,
  'token': 'd77029ce-42d3-4e6f-ab7a-8afd9edf2504',
  'email': '',
  'created': {'raw': 1699500252, 'formatted': '09/11/2023'},
  'last_activity': None,
  'options': None,
  'download_url': 'https://filesender.aarnet.edu.au/?s=download&token=d77029ce-42d3-4e6f-ab7a-8afd9edf2504',
  'errors': []}]

In [ ]:

try:
    for f in transfer['files']:
        path = files[f['name']+':'+str(f['size'])]['path']
        size = files[f['name']+':'+str(f['size'])]['size']
        # putChunks
        if debug:
            print('putChunks: '+path)
        with open(path, mode='rb', buffering=0) as fin:
            for offset in range(0, size, upload_chunk_size):
                if progress:
                    print('Uploading: '+path+' '+str(offset)+'-'+str(min(offset +
                          upload_chunk_size, size))+' '+str(round(offset/size*100))+'%')
                data = fin.read(upload_chunk_size)
                # print(data)
                putChunk(transfer, f, data, offset)

        # fileComplete
        if debug:
            print('fileComplete: '+path)
        fileComplete(transfer, f)
        if progress:
            print('Uploading: '+path+' '+str(size)+' 100%')

    # transferComplete
    if debug:
        print('transferComplete')
    transferComplete(transfer)
    if progress:
        print('Upload Complete')

except Exception as inst:
    print(type(inst))
    print(inst.args)
    print(inst)

    # deleteTransfer
    if debug:
        print('deleteTransfer')
    deleteTransfer(transfer)
